In [1]:
import os, sys
from pathlib import Path

# Lên 1 cấp so với cwd để về folder project root
project_root = Path.cwd().parent   # nếu cwd là project/notebooks
sys.path.append(str(project_root))

# Thiết lập BASE_DIR tương tự
BASE_DIR = project_root

print("Project root:", project_root)
print("BASE_DIR:", BASE_DIR)

Project root: e:\twit_sentiment_analysis
BASE_DIR: e:\twit_sentiment_analysis


In [2]:
import pandas as pd
import logging

In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
from src.models.transformer_manager import ModelManager

c:\Users\nn200\miniconda3\envs\twit-sentiment-group-2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model_name = 'modernbert'
model_path=f"{BASE_DIR}/mlflow/artifact/models/{model_name}"
model_dir = Path(model_path).resolve() 
version = 1
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
)
    # Try loading local directory
model_obj = AutoModelForSequenceClassification.from_pretrained(
        model_dir,
        local_files_only=True,
        device_map='cuda:0' if torch.cuda.is_available() else 'cpu'
    )
tokenizer_obj = AutoTokenizer.from_pretrained(
        model_dir,
        local_files_only=True,
        device_map='cuda:0' if torch.cuda.is_available() else 'cpu'
    )
manager = ModelManager(
        model_name=model_name,
        model_path=model_dir,
        finetune=False,
        model_obj=model_obj,
        tokenizer_obj=tokenizer_obj)

In [6]:
texts = [
    "I absolutely love this product! It works like a charm.",
    "I'm really disappointed. The service was terrible and slow."
]


prediction = manager.predict_batch(texts)
prediction

[2, 0]

# Pull data

In [7]:
from config.dataClient import get_db

In [8]:
db = get_db()
# 1) Lấy datetime lớn nhất
rows = db.read(
    table_name="TWEET_STAGING",
    columns=["tweet_id", "topic", "content", "created_at"],
    conditions={"moved_to_product": False}
)

if not rows:
    logging.info("No new records to process.")
else:
    # 2. Lấy content để batch predict
    contents = [row[2] for row in rows]
    predicted_sentiments = manager.predict_batch(contents)

    # 3. Tạo dữ liệu để insert
    data_to_insert = [
        {
            "staging_tweet_id": row[0],
            "topic": row[1],
            "predicted_sentiment": pred,
            "created_at": row[3]
        }
        for row, pred in zip(rows, predicted_sentiments)
    ]

    # 4. Gọi batch insert với column mapping rõ ràng
    column_mapping = {
        "staging_tweet_id": "staging_tweet_id",
        "topic": "topic",
        "predicted_sentiment": "predicted_sentiment",
        "created_at": "created_at"
    }

    db.batch_insert("TWEET_PRODUCT", data_to_insert, column_mapping)

INFO:root:Successfully inserted 44 records into TWEET_PRODUCT.
